<a href="https://colab.research.google.com/github/oneonlee/NLP-author-classification/blob/main/LGE/plus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

원래 코드에 합치면 과도한 메모리 사용 등으로 예기치 못한 종료가 일어나 오류가 발생한다는 메세지가 떠서 따로 첨부했다.

주제 간 거리 지도

In [2]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 69.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.3 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.3 which is incompatibl

In [9]:
nltk.download('punkt')
nltk.download('wordnet')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [5]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import gensim
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.tokenize import word_tokenize
import pyLDAvis.gensim

# Load data
path = '/content/drive/MyDrive/'
train_df = pd.read_csv(path+'train.csv')
test_df = pd.read_csv(path+'test_x.csv')
submission_df = pd.read_csv(path+'/sample_submission.csv')

train_df['text_l'] = train_df['text'].str.lower() 
test_df['text_l'] = test_df['text'].str.lower() 

NUM_TOPICS = 5

def get_lda_objects(text):
    nltk.download('stopwords')    
    stop=set(stopwords.words('english'))

    
    def _preprocess_text(text):
        corpus=[]
        stem=PorterStemmer()
        lem=WordNetLemmatizer()
        for news in text:
            words=[w for w in word_tokenize(news) if (w not in stop)]

            words=[lem.lemmatize(w) for w in words if len(w)>2]

            corpus.append(words)
        return corpus
    
    corpus=_preprocess_text(text)
    
    dic=gensim.corpora.Dictionary(corpus)
    bow_corpus = [dic.doc2bow(doc) for doc in corpus]
    
    lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = NUM_TOPICS, 
                                   id2word = dic,                                    
                                   passes = 10,
                                   workers = 2)
    
    return lda_model, bow_corpus, dic

def plot_lda_vis(lda_model, bow_corpus, dic):
    pyLDAvis.enable_notebook()
    vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dic)
    return vis

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
lda_model, bow_corpus, dic = get_lda_objects(train_df['text_l'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
lda_model.show_topics()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.071*"odin" + 0.039*"said" + 0.016*"asked" + 0.014*"sir" + 0.012*"mr." + 0.011*"man" + 0.009*"cried" + 0.009*"face" + 0.009*"hand" + 0.008*"voice"'),
 (1,
  '0.024*"odin" + 0.011*"could" + 0.010*"would" + 0.009*"one" + 0.006*"time" + 0.005*"even" + 0.005*"though" + 0.005*"might" + 0.004*"made" + 0.004*"first"'),
 (2,
  '0.017*"odin" + 0.011*"upon" + 0.008*"hand" + 0.007*"one" + 0.007*"door" + 0.006*"eye" + 0.006*"face" + 0.006*"came" + 0.006*"back" + 0.006*"room"'),
 (3,
  '0.076*"odin" + 0.029*"said" + 0.018*"mr." + 0.012*"know" + 0.009*"would" + 0.009*"well" + 0.009*"say" + 0.009*"think" + 0.008*"miss" + 0.008*"come"'),
 (4,
  '0.015*"said" + 0.014*"n\'t" + 0.012*"know" + 0.012*"one" + 0.011*"man" + 0.011*"would" + 0.011*"come" + 0.010*"cried" + 0.008*"odin" + 0.008*"say"')]

In [12]:
plot_lda_vis(lda_model, bow_corpus, dic)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.150882 -0.014018       1        1  29.383779
2     -0.013900 -0.251402       2        1  24.316926
3     -0.076238  0.136462       3        1  24.118795
4     -0.028807  0.100465       4        1  13.933326
0      0.269826  0.028493       5        1   8.247173, topic_info=         Term          Freq         Total Category  logprob  loglift
13       odin  38433.000000  38433.000000  Default  30.0000  30.0000
40       said  12994.000000  12994.000000  Default  29.0000  29.0000
34        mr.   5821.000000   5821.000000  Default  28.0000  28.0000
455      know   4780.000000   4780.000000  Default  27.0000  27.0000
510       n't   2052.000000   2052.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
133      upon    349.907324   5279.880880   Topic5  -5.5196  -0.2187
671  answered    228.549247   1025.848307   Topic5  -5.9455   0.9938
50     doctor    226.921300   1074.918470   Topic5  -5.9526   0.9399
398      word    247.186996   2500.677937   Topic5  -5.8671   0.1811
624      like    240.748655   3922.630644   Topic5  -5.8935  -0.2955

[418 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
11613      5  0.981079    'come
600        4  0.995096      'do
4748       4  0.995512      'em
4321       5  0.995477      'he
9250       5  0.977802     'how
...      ...       ...      ...
557        2  0.114260    young
557        3  0.318670    young
557        4  0.086481    young
557        5  0.167197    young
1292       3  0.989457  zametov

[866 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 4, 5, 1])

In [13]:
lda_model, bow_corpus, dic = get_lda_objects(test_df['text_l'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
plot_lda_vis(lda_model, bow_corpus, dic)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.100239  0.099627       1        1  30.189173
0      0.104634 -0.007959       2        1  21.222974
1     -0.137158  0.086854       3        1  20.749029
3      0.108119 -0.100832       4        1  15.245251
4     -0.175835 -0.077691       5        1  12.593573, topic_info=      Term          Freq         Total Category  logprob  loglift
67    odin  23593.000000  23593.000000  Default  30.0000  30.0000
41    know   3280.000000   3280.000000  Default  29.0000  29.0000
281   said   4924.000000   4924.000000  Default  28.0000  28.0000
871    ...   1154.000000   1154.000000  Default  27.0000  27.0000
517  shall   1542.000000   1542.000000  Default  26.0000  26.0000
..     ...           ...           ...      ...      ...      ...
28     way    265.215522   2062.803124   Topic5  -5.9404   0.0207
141    set    219.115735    765.421125   Topic5  -6.1314   0.8212
47   would    266.857925   5767.299606   Topic5  -5.9343  -1.0013
811  night    228.219942   1301.476220   Topic5  -6.0907   0.3310
449    see    233.479176   2708.144530   Topic5  -6.0679  -0.3789

[471 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1838      2  0.231752      'll
1838      4  0.766363      'll
871       1  0.000866      ...
871       2  0.101345      ...
871       4  0.898250      ...
...     ...       ...      ...
166       2  0.345854      yet
166       3  0.155463      yet
166       4  0.126014      yet
166       5  0.132863      yet
7647      4  0.988370  zametov

[1132 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2, 4, 5])